* Import packages.
* Define platform API endpoint (modify for your environment).
* Define platform API key (found on user page).
* Create platform API client.

In [ ]:
import os
import yb_platform_client
from yb_platform_client.api import provider_api, session_api
from yb_platform_client.model.provider import Provider
from yb_platform_client.model.region import Region
from pprint import pprint

platform_address = 'http://localhost:9000'
platform_api_key = os.getenv('YB_API_KEY')

api_client = yb_platform_client.ApiClient(yb_platform_client.Configuration(
    host = platform_address,
    api_key = {
        'apiKeyAuth': platform_api_key,
    }
))

Make an API call to session endpoint to determine customer UUID.

In [ ]:
session_api = session_api.SessionApi(api_client)

try:
    session_info = session_api.get_session_info()
except yb_platform_client.ApiException as e:
    print("Error get_session_info: %s" % e)
    raise

customer_uuid = session_info.get('customer_uuid')
print('Customer UUID:\n%s' % customer_uuid)

Define new provider object.

In [ ]:
new_provider = Provider(
    air_gap_install=False,
    code="gcp",
    config={
        "GCE_EMAIL": "<service acct email>",
        "GCE_HOST_PROJECT": "<gcp project>",
        "GCE_PROJECT": "<gcp project>",
        "YB_FIREWALL_TAGS": "<vpc tags>",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "client_email": "<service account email>",
        "client_id": "<service account id>",
        "client_x509_cert_url": "<service account cert url>",
        "private_key": "<service account key>",
        "private_key_id": "<service account key id>",
        "project_id": "<gcp project>",
        "token_uri": "https://accounts.google.com/o/oauth2/token",
        "type": "service_account",
    },
    dest_vpc_id="yugabyte-network",
    name="api-test",
    regions=[
        Region(
            code="us-central1",
            name="us-central1",
            zones=[],
        ),
    ],
    ssh_port=54422,
)

Make API call to provider endpoint to create new provider.

In [ ]:
provider_api = provider_api.ProviderApi(api_client)

try:
    api_response = provider_api.create_providers(customer_uuid, new_provider)
except yb_platform_client.ApiException as e:
    print('Error create_providers: %s' % e)
    raise

pprint(api_response)